Contained in this notebook is the individual analysis of one tcx file, and how to obtain the split paces, over a given distance (would need to pay for the MVP subscription to do this alternatively :-) ) 

In [2]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

tree = ET.parse('../data/8.03mi Run.tcx')

In [19]:
def parse_run_file(tcx,plots=True):
    """Parse the race file for useful split information, as well as potentially valuable plots. 
    
    
    Parameters
    ----------
    
    tcx : str
        The path to the tcx file to be parsed. 
        
        
    Returns
    -------
    
    
    """
    
    #import dependencies and plotting preferences. 
    
    import pandas as pd
    import numpy as np
    import xml.etree.ElementTree as ET
    import matplotlib.pyplot as plt
    plt.style.use('fivethirtyeight')
    %matplotlib inline

    #bring in tcx file.. note it is in an xml format, so this had to be searched a little before
    #I found the useful information. 
    tree = ET.parse(tcx)
    root = tree.getroot()
    #travel down the tree to get the distance measurements. 
    activity = root.getchildren()[0]
    try:
        measures = activity.getchildren()[0].getchildren()[1].getchildren()[4].getchildren()[1:]
    except:
        print("Are you sure you didn't do this workout on a treadmill? No tracking information available.")
        return
    measures[1].getchildren()[0].text
    distances = []
    times = []
    for i,point in enumerate(measures):
        if len(point.getchildren()) == 4:
            dist = point.getchildren()[-1]
            distances.append(float(dist.text))
            times.append(point.getchildren()[0].text)
            
    #create a dataframe for this run, and generate a useful time series. 
    time_df = pd.DataFrame([])
    time_df['Time'] = times
    time_df['Time'] = time_df.Time.str.split('T',expand = True)[1]
    time_df['Time'] = time_df['Time'].str.split('+',expand=True)[0]
    time_df = time_df['Time'].str.split(':',expand=True)
    time_df = time_df.astype('float')
    time_df['Time Elapsed'] = time_df[0]*3600 + time_df[1]*60 + time_df[2]
    time_df['Time Elapsed'] = time_df['Time Elapsed'] -time_df['Time Elapsed'].values[0]
    del time_df[0],time_df[1],time_df[2]
    time_df['distance_meters'] = distances
    
    if plots:
        plt.figure()
        plt.title('Distance vs. Time')
        plt.xlabel('Time Elapsed (s)')
        plt.ylabel('Distance (m)')
        plt.plot(time_df['Time Elapsed'], time_df['distance_meters'])
    #create column for miles travelled. 
    time_df['distance_miles'] = time_df['distance_meters'] / 1609.344
    #round off values. 
    time_df['distance_miles'] = [("%.1f" % a) for a in time_df['distance_miles'].values]
    time_df['Time Elapsed'] = [("%.1f" % a) for a in time_df['Time Elapsed'].values]

    #now get pace information
    pace_data = pd.concat([g.tail(n=1) for _,g in time_df.groupby('distance_miles')])
    
    split_paces = []
    for i,_ in enumerate(pace_data['Time Elapsed'].values):
        if i < (len(pace_data)-1):
            time_elapsed = (float(pace_data['Time Elapsed'].values[i+1])
                            - float(pace_data
                                    ['Time Elapsed'].values[i]))
                
            sec_per_mile = 10 * time_elapsed
            min_per_mile = sec_per_mile/60
            split_paces.append(min_per_mile)
        else:
            split_paces.append(None)
    
    pace_data['min per mile split'] = split_paces 
    pace_data['rolling_mean_splits'] = pace_data['min per mile split'].rolling(window=5).mean()

    if plots: 
        plt.figure()
        plt.title('Split Pace vs. Time Elapsed')
        plt.xlabel('Time Elapsed (s)')
        plt.ylabel('Pace (min/mi)')
        plt.ylim((20,0))
        plt.plot(pace_data['Time Elapsed'],pace_data['min per mile split'],label='.1 mile splits')
        plt.plot(pace_data['Time Elapsed'],pace_data['rolling_mean_splits'],label='.5 mile avg') #half mile rolling mean. 
        plt.legend()

    return pace_data


In [20]:
parse_run_file('../data/3.50mi Run.tcx')

Are you sure you didn't do this workout on a treadmill? No tracking information available.


> <ipython-input-8-b9ab9577bcd5>(33)parse_run_file()
     31     #travel down the tree to get the distance measurements.
     32     activity = root.getchildren()[0]
---> 33     measures = activity.getchildren()[0].getchildren()[1].getchildren()[4].getchildren()[1:]
     34     measures[1].getchildren()[0].text
     35     distances = []

ipdb> measures
*** NameError: name 'measures' is not defined
ipdb> activity
<Element '{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Activities' at 0x11408abd8>
ipdb> activity.getchildren(
*** SyntaxError: unexpected EOF while parsing
ipdb> activity.getchildren()
[<Element '{http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2}Activity' at 0x113320cc8>]
ipdb> exit


In [ ]:
#travel down the tree to get the distance measurements. 
activity = root.getchildren()[0]
measures = activity.getchildren()[0].getchildren()[1].getchildren()[4].getchildren()[1:]
measures[1].getchildren()[0].text
distances = []
times = []
for i,point in enumerate(measures):
    if len(point.getchildren()) == 4:
        dist = point.getchildren()[-1]
        distances.append(float(dist.text))
        times.append(point.getchildren()[0].text)

In [ ]:
#create a dataframe for this run
time_df = pd.DataFrame([])
time_df['Time'] = times
time_df['Time'] = time_df.Time.str.split('T',expand = True)[1]
time_df['Time'] = time_df['Time'].str.split('+',expand=True)[0]
time_df = time_df['Time'].str.split(':',expand=True)
time_df = time_df.astype('float')
time_df['Time Elapsed'] = time_df[0]*3600 + time_df[1]*60 + time_df[2]
time_df['Time Elapsed'] = time_df['Time Elapsed'] -time_df['Time Elapsed'].values[0]
del time_df[0],time_df[1],time_df[2]
time_df['distance_meters'] = distances

In [ ]:
#create column for miles travelled. 
time_df['distance_miles'] = time_df['distance_meters'] / 1609.344

Now I would like to calculate my split pace for every .1 miles.



In [ ]:
time_df['distance_miles'] = [("%.1f" % a) for a in time_df['distance_miles'].values]
time_df['Time Elapsed'] = [("%.1f" % a) for a in time_df['Time Elapsed'].values]

In [ ]:
pace_data = pd.concat([g.tail(n=1) for _,g in time_df.groupby('distance_miles')])

In [ ]:
pace_data

In [ ]:
split_paces = []
for i,_ in enumerate(pace_data['Time Elapsed'].values):
    if i < (len(pace_data)-1):
        time_elapsed = (float(pace_data['Time Elapsed'].values[i+1])
                            - float(pace_data
                                    ['Time Elapsed'].values[i]))
        
        dist = .1
        
        sec_per_mile = 10 * time_elapsed
        min_per_mile = sec_per_mile/60
        split_paces.append(min_per_mile)
    else:
        split_paces.append(None)

In [ ]:
pace_data['min per mile split'] = split_paces 

In [ ]:
plt.title('Split Pace vs. Time Elapsed')
plt.ylim((20,0))
plt.plot(pace_data['Time Elapsed'],pace_data['min per mile split'])
plt.plot(pace_data['Time Elapsed'],pace_data['min per mile split'].rolling(window=5).mean()) #half mile rolling mean. 


In [ ]:
plt.ylim((20,0))

plt.plot(pace_data['Time Elapsed'],pace_data['min per mile split'].rolling(window=5).mean()) #half mile rolling mean. 



In [ ]:
for child in activity:
    print('   ',child.attrib)

In [ ]:
import tcxparser
tcx = tcxparser.TCXParser('8.03mi Run.tcx')

In [ ]:
txc

In [ ]:
#duration in seconds. 
tcx.time_values()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
#plt.plot(tcx.altitude_points())
tcx.distance

In [ ]:
3390 / 60

In [ ]:
#spot check passes. This is true!


tcx.activity.getchildren()

In [ ]:
root = tree.getroot()


In [ ]:
root.findtext()

In [ ]:
for child in tree:
    print(child.tag, child.attrib)

In [ ]:
root.tag

In [ ]:
root.getchildren()

In [ ]:
tree.findtext

In [ ]:
import untangle
obj = untangle.parse('8.03mi Run.tcx')

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'8.03mi Run.tcx'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.


In [ ]:
import os, csv
import lxml.etree as ET

# SET DIRECTORY
#cd = os.path.dirname(os.path.abspath(__file__))

# LOAD XML FILE
xmlfile = 'trackXML.xml'
dom = ET.parse(os.path.join(cd, xmlfile))

# DEFINING COLUMNS
columns = ['latitude', 'longitude', 'altitude', 'distance', 'watts']

# OPEN CSV FILE
with open(os.path.join(cd,'trackData.csv'), 'w') as m:
    writer = csv.writer(m)    
    writer.writerow(columns)

    nodexpath = dom.xpath('//Trackpoint')

    dataline = []   # FOR ONE-ROW CSV APPENDS
    datalines = []  # FOR FINAL OUTPUT 
    for j in range(1,len(nodexpath)+1):        
        dataline = []

        # LOCATE PATH OF EACH NODE VALUE
        latitudexpath = dom.xpath('//Trackpoint[{0}]/Position/LatitudeDegrees/text()'.format(j))
        dataline.append('') if latitudexpath == [] else dataline.append(latitudexpath[0])

        longitudexpath = dom.xpath('//Trackpoint[{0}]/Position/LongitudeDegrees/text()'.format(j))
        dataline.append('') if longitudexpath == [] else dataline.append(longitudexpath[0])

        altitudexpath = dom.xpath('//Trackpoint[{0}]/AltitudeMeters/text()'.format(j))
        dataline.append('') if altitudexpath == [] else dataline.append(altitudexpath[0])

        distancexpath = dom.xpath('//Trackpoint[{0}]/DistanceMeters/text()'.format(j))
        dataline.append('') if distancexpath == [] else dataline.append(distancexpath[0])

        wattsxpath = dom.xpath("//Trackpoint[{0}]/*[name()='Extensions']/*[name()='TPX']/*[name()='Watts']/text()".format(j))
        dataline.append('') if wattsxpath == [] else dataline.append(wattsxpath[0])

        datalines.append(dataline)
        writer.writerow(dataline)

print(datalines)